In [11]:
from battle import *
from entities.trainer import *
from entities.pokemon import *
from moves.moves import *
from globals import *
from entities.team import *
from events.items import *
from events.statuses import *
from events.weathers import *
from battle.battle import *

In [12]:
moves=[Tackle(), Freeze(), Thunder(), Burn(), Earthquake(), ThunderWave(), Rest(), RainDance(), WaterLance(), SwordsDance(), FireBall(), FireSwipe(), Conflagration()]
pikachuStats=Stats(200,100,100,100,100,100)
pikachus=[Pikachu('pika1', 15, pikachuStats, moves)]#, Pikachu('pika2', 15, pikachuStats, moves), Pikachu('pika3', 15, pikachuStats, moves), Pikachu('pika4', 15, pikachuStats, moves)]

ratStats=Stats(200,100,100,100,100,101)
rats=[Rattata('rat1', 20, ratStats, moves)]#, Rattata('rat2', 20, ratStats, moves), Rattata('rat3', 20, ratStats, moves), Rattata('rat4', 20, ratStats, moves)]
rats[0].item=Sword(rats[0])

bellStats=Stats(200, 75, 100, 150, 150, 30)
bell=Bell('Bong', 15, bellStats, moves, None, None)

shroomHogStats=Stats(150, 100, 75, 150, 150, 75)
shroomHog=Shroomhog('SadHog', 15, shroomHogStats, moves)
shroomHog.item=Sword(shroomHog)
t1=Trainer('Trainer 1', [bell, pikachus[0]], None)
t2=Trainer('Trainer 2', [rats[0], shroomHog], None)
teams=[Team('Team Bell', [t1], 1), Team('Team Shrooms', [t2], 1)]
testBattle=Battle(teams)

In [13]:
testBattle.runBattle()
# TODO: sync pokemon fainted status with corresponding slot being empty
# TODO: only show trainers when swapping and sending out pokemon if they have pokemon available

BEING SCHEDULED
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 1234
BEING SCHEDULED
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 1234


AttributeError: module '__main__' has no attribute '__file__'

In [ ]:
from eventQueue.eventQueue import *
from collections import defaultdict
from collections.abc import Iterable

class EventSystem:
    """Event System that handles the scheduling and triggering of events. Allows
    for adding of permanent and temporary events with duration and proc lifetimes.
    Duration, start turns, and proc lifetimes are handled inside the Event class.

    Attributes:
        scheduledEvents (dict [Trigger: list[Event]]): Dictionary that keeps track of which temporary events are mapped to which trigger.
        permanentEvents (dict [Trigger: list[Event]]): Dictionary that keeps track of which permanent events are mapped to which trigger.
    """
    def __init__(self):
        self.scheduledEvents=defaultdict(list)
        self.permanentEvents=defaultdict(list)

    def addPermanentEvent(self, event, triggers=None):
        """Adds a permanent event to the EventSystem that will be fired when the triggers are activated.
        If no triggers are passed in, the event's default triggers will be used.

        Arguments:
            event (Event): Event to be added as a permanent event
            triggers (list[Trigger] or Trigger): Trigger/s to add event to
        """
        if triggers is None:
            triggers=event.triggers
        if not isinstance(triggers, Iterable):
            triggers=[triggers]

        for trigger in triggers:
            self.permanentEvents[trigger].append(event)

    def removePermanentEvent(self, shouldRemoveFn, triggers=None):
        """Removes all permanent events from the specified triggers that satisfy the removal function.
        If no triggers are passed in, the event's default triggers will be used.

        Arguments:
            shouldRemoveFn (Callable[[Event], bool]): Function used to determine whether to remove event or not
            triggers (list[Trigger] or Trigger): Trigger/s to remove events from
        """
        if triggers is None:
            triggers=self.scheduledEvents.keys()
        if not isinstance(triggers, Iterable):
            triggers=[triggers]

        for trigger in triggers:
            self.permanentEvents[trigger]=[e for e in self.permanentEvents[trigger] if shouldRemoveFn(e)]

    def schedule(self, event, battleContext, triggers=None):
        """Schedules a temporary event to the EventSystem that will be fired when the triggers are activated.
        If no triggers are passed in, the event's default triggers will be used. Events will only be triggered
        when the turn is in the range [startTurn, startTurn+duration) of the event.

        Arguments:
            event (Event): Event to be added as a temporary event
            triggers (list[Trigger] or Trigger): Trigger/s to add event to
        """
        if triggers is None:
            triggers=event.triggers
        if not isinstance(triggers, Iterable):
            triggers=[triggers]

        if event.startTurn+event.duration-1<battleContext.turn:
            return
        
        for trigger in triggers:
            self.scheduledEvents[trigger].append(event)

    def unschedule(self, shouldRemoveFn, triggers=None):
        """Removes all temporary events from the specified triggers that satisfy the removal function.
        If no triggers are passed in, the event's default triggers will be used.

        Arguments:
            shouldRemoveFn (Callable[[Event], bool]): Function used to determine whether to remove event or not
            triggers (list[Trigger] or Trigger): Trigger/s to remove events from
        """
        if triggers is None:
            triggers=self.scheduledEvents.keys()
        if not isinstance(triggers, Iterable):
            triggers=[triggers]
            
        for trigger in triggers:
            self.scheduledEvents[trigger]=[e for e in self.scheduledEvents[trigger] if shouldRemoveFn(e)]

    def remove(self, shouldRemoveFn, triggers=None):
        """Removes all temporary and permanent events from the specified triggers that satisfy the removal function.
        If no triggers are passed in, the event's default triggers will be used.
        
        Arguments:
            shouldRemoveFn (Callable[[Event], bool]): Function used to determine whether to remove event or not
            triggers (list[Trigger] or Trigger): Trigger/s to remove events from
        """
        self.removePermanentEvent(shouldRemoveFn=shouldRemoveFn, triggers=triggers)
        self.unschedule(shouldRemoveFn=shouldRemoveFn, triggers=triggers)

    def _populateQueue(self, eventQueue, battleContext, trigger):
        """Populates eventQueue with all events that should be added in for the trigger specified.
        
        Arguments:
            eventQueue (EventQueue): EventQueue to populate
            battleContext (BattleContext): The context of the battle
            trigger (Trigger): The trigger to populate the queue for
        """
        for event in self.scheduledEvents[trigger]:
            if event.startTurn<=battleContext.turn<=event.startTurn+event.duration-1:
                eventQueue.push(event, trigger)

        for event in self.permanentEvents[trigger]:
            eventQueue.push(event, trigger)

    def trigger(self, battleContext, eventContext, trigger):
        """Triggers the specified event type, evaluating all active scheduled and permanent events.
        Each event may modify the battle or event context. Events with depleted procs are removed.

        Arguments:
            battleContext (BattleContext): The context of the battle
            eventContext (EventContext): Context relating to the events. Events modify this to interact with outside code.
            trigger (Trigger): The trigger to activate
        """
        currTurn=battleContext.turn
        eq=EventQueue()
        self._populateQueue(eventQueue=eq, battleContext=battleContext, trigger=trigger)

        while not eq.empty():
            event=eq.top()
            eq.pop()
            if event.trigger(battleContext=battleContext, eventContext=eventContext, trigger=trigger):
                event.procs-=1
                if event.procs<=0:
                    self.remove(lambda x: x.id==event.id, event.triggers)

        if trigger==Trigger.END_TURN:
            for trig, events in self.scheduledEvents.items():
                self.scheduledEvents[trig]=[e for e in events if e.startTurn+e.duration-1>currTurn]